In [ ]:
#identigying customers that havent bought in the last three month

import pandas as pd
import datetime as dt

data = pd.read_csv("rfm_xmas19.txt", parse_dates=["trans_date"])
group_by_customer = data.groupby("customer_id")
last_transaction = group_by_customer["trans_date"].max()

best_churn = pd.DataFrame(last_transaction) 
cutoff_day = dt.datetime(2019,10,16) # date to interpret cutoff point
best_churn['churned'] = best_churn['trans_date'].apply(lambda h: 1 if h < cutoff_day else 0)

In [ ]:
# ranking customers according to a threshold
#using a waithed sum model

best_churn["nr_of_transactions"] = group_by_customer.size() #the number of transactions each customer made
best_churn['amount_spent'] = group_by_customer.sum() # the amount of money each customer spent
best_churn.drop('trans_date', axis = 1, inplace = True)


In [ ]:
#scale the amount spent and the number of transactions with Min-Max feature Sacling
best_churn["scaled_tran"] = (best_churn["nr_of_transactions"] \
                             - best_churn["nr_of_transactions"].min()) \
                             / (best_churn["nr_of_transactions"].max() \
                             - best_churn["nr_of_transactions"].min())

best_churn["scaled_amount"] = (best_churn["amount_spent"] \
                               -best_churn["amount_spent"].min()) \
                               / (best_churn["amount_spent"].max() \
                               - best_churn["amount_spent"].min())

best_churn["score"] = 100*(.5*best_churn["scaled_tran"] \
                           + .5*best_churn["scaled_amount"])

best_churn.sort_values("score", inplace=True, ascending=False)
